In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
# test connetion
!ls '/content/gdrive/My Drive'

In [ ]:
!pip install ultralytics

In [ ]:
# test connetion
!ls '/content/gdrive/My Drive/Netlist generator'
!ls '/content/gdrive/My Drive/Netlist generator/Testing'
!ls '/content/gdrive/My Drive/Netlist generator/Training'
!ls '/content/gdrive/My Drive/Netlist generator/Testing/Model testing'

**Run model on all test images**

In [ ]:
import os
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

# Paths for model and image folder
model_path = '/content/gdrive/My Drive/Netlist generator/Current trained model/pose/train/weights/last.pt'
image_folder = '/content/gdrive/My Drive/Netlist generator/Testing/Model testing/Test images/'

# Load YOLO model
model = YOLO(model_path)

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# List to store processed images
processed_images = []

# Loop through each image file
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)

    # Load image
    img = cv2.imread(image_path)

    # Run inference
    results = model(image_path)[0]

    # Process each result
    for result in results:
        # Handle multiple detections properly
        for cls, keypoints, bbox in zip(result.boxes.cls.cpu().numpy(),
                                         result.keypoints.xy.cpu().numpy(),
                                         result.boxes.xyxy.cpu().numpy()):
            class_idx = int(cls)  # Convert single class index to int
            object_name = results.names[class_idx]  # Get the class name using the class index

            # Extract bounding box coordinates
            x_min, y_min, x_max, y_max = map(int, bbox)

            # Draw a rectangle around the detected object
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

            # Display object name above the bounding box
            cv2.putText(img, object_name, (x_min, y_min),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 255), 1)

            for keypoint in keypoints:
              x, y = int(keypoint[0]), int(keypoint[1])  # Explicitly get x, y coordinates
              cv2.circle(img, (x, y), radius=4, color=(0, 255, 0), thickness=-1)

    # Append processed image to list
    processed_images.append((image_file, img))

# Display all processed images
for file_name, processed_img in processed_images:
    print(f"Displaying image: {file_name}")
    cv2_imshow(processed_img)

**Run model on one test image**

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

# Paths for model and image
test_image_name = 'circuit_test_0002.jpg'
model_path = '/content/gdrive/My Drive/Netlist generator/Current trained model/pose/train/weights/last.pt'
image_folder = '/content/gdrive/My Drive/Netlist generator/Testing/Model testing/Test images/' + test_image_name

# Load image
img = cv2.imread(image_path)

# Load YOLO model
model = YOLO(model_path)

# Run inference
results = model(image_path)[0]

# Process each result
for result in results:
    keypoints = result.keypoints.xy.cpu().numpy()[0]  # Move tensor to CPU before converting to NumPy
    class_idx = int(result.boxes.cls.cpu().numpy())  # Convert tensor to numpy, then to int
    object_name = results.names[class_idx]    # Get the class name using the class index

    # Draw bounding box around the object
    bbox = result.boxes.xyxy.cpu().numpy()[0]  # Move tensor to CPU before converting to NumPy
    x_min, y_min, x_max, y_max = map(int, bbox)  # Extract bounding box coordinates

    # Draw a rectangle (square) around the detected object
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

    # Display object name above the bounding box
    cv2.putText(img, object_name, (x_min , y_min ),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 255), 1)
    
    # Draw keypoints on image
    for keypoint in keypoints:
      x, y = int(keypoint[0]), int(keypoint[1])  # Explicitly get x, y coordinates
      cv2.circle(img, (x, y), radius=4, color=(0, 255, 0), thickness=-1)

# Display image with keypoints and bounding box
cv2_imshow(img)